In [1]:
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader
from langchain_unstructured import UnstructuredLoader
from langchain_core.documents import Document
from typing import List 
from langchain_community.document_loaders import Docx2txtLoader

from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_experimental.text_splitter import SemanticChunker
import weaviate 
from weaviate.classes.config import Configure, Property, DataType
from weaviate.util import generate_uuid5
from tqdm import tqdm
from dotenv import load_dotenv
load_dotenv()


True

# 1. Load document (texe, PDF, Markdown, Web,..)

In [2]:
def load_documents_format(file_path: str) -> List[Document]:
    """
    Load documents using the best loader for each file format.
        
    Returns:
        List of Document objects with page_content and metadata
    """
    path = Path(file_path)
    file_ext = path.suffix.lower()
    
    # PDF files - one Document per page
    if file_ext == '.pdf':
        loader = PyPDFLoader(file_path)
        docs = loader.load()
        return docs
    
    # Word documents - preserves formatting
    elif file_ext == '.docx':
        loader = Docx2txtLoader(file_path)
        docs = loader.load()
        return docs
    
    # Text/markdown files
    elif file_ext == '.txt' or file_ext == '.md':
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        return [Document(
            page_content=content,
            metadata={
                "source": str(path),
                "file_type": "text"
            }
        )]
    
def load_multiple_documents(
    directory_path: str,
    file_types: List[str] = None,
    verbose: bool = False
) -> List[Document]:
    """
    Load all documents from a directory using optimal loaders.
        
    Returns:
        List of all loaded Documents
    """
    if file_types is None:
        file_types = ['.pdf', '.docx', '.txt', '.md']
    
    directory = Path(directory_path)
    all_documents = []
    
    for file_type in file_types:
        for file_path in directory.glob(f'*{file_type}'):
            try:
                if verbose:
                    print(f"Loading {file_path.name}...")
                docs = load_documents_format(str(file_path))
                
                all_documents.extend(docs)
                if verbose:
                    print(f"✓ Loaded {len(docs)} documents")
            except Exception as e:
                print(f"✗ Error loading {file_path.name}: {e}")
    
    return all_documents

In [3]:
documents = load_multiple_documents('documents/', verbose=True)

Loading Privacy Policy f... (1).docx...
✓ Loaded 1 documents
Loading SaaS Agreement f... (1).docx...
✓ Loaded 1 documents


In [4]:
documents[0].metadata

{'source': 'documents\\Privacy Policy f... (1).docx'}

In [5]:
print(f'Number of documents: {len(documents)}')

Number of documents: 2


In [6]:
for doc in documents[:3]:
    print(f'Content: {doc.page_content}')  
    print(f'Metadata: {doc.metadata}')
    print('-' * 80)

Content: Chính sách Quyền riêng tư của TechNova Solutions



Cập nhật lần cuối: Ngày 20 tháng 12 năm 2025



Chào mừng bạn đến với TechNova Solutions ("Công ty," "chúng tôi," hoặc "của chúng tôi"). Chúng tôi cam kết bảo vệ thông tin cá nhân và quyền riêng tư của bạn. Nếu bạn có bất kỳ câu hỏi hoặc lo ngại nào về chính sách của chúng tôi, hoặc về các thực tiễn của chúng tôi liên quan đến thông tin cá nhân của bạn, vui lòng liên hệ với chúng tôi tại privacy@technovasolutions.io.



Khi bạn truy cập trang web của chúng tôi https://www.technovasolutions.io, sử dụng ứng dụng di động của chúng tôi, hoặc sử dụng bất kỳ dịch vụ nào của chúng tôi (gọi chung là "Dịch vụ," bao gồm Trang web và Ứng dụng), chúng tôi đánh giá cao việc bạn đã tin tưởng giao phó thông tin cá nhân của mình cho chúng tôi. Chúng tôi rất coi trọng quyền riêng tư của bạn. Trong thông báo quyền riêng tư này, chúng tôi cố gắng giải thích cho bạn một cách rõ ràng nhất có thể về những thông tin chúng tôi thu thập, cách chúng t

# 3. Chunking
    - Fixed 
    - Recursive ("\n\n", "\n", " ", "")
    - Semantic
    - Markdown character
    - LLM-based (context-aware)

In [7]:
def chunk_documents(
    documents: List[Document],
    strategy: str = 'recursive',
    chunk_size: int = 500,
    chunk_overlap: int = 50
) -> List[Document]:
        
        chunks = []
        if strategy == 'fixed':
            text_splitter = CharacterTextSplitter(
                chunk_size=chunk_size, 
                chunk_overlap=chunk_overlap
            )
            chunks = text_splitter.split_documents(documents)
            
        elif strategy == "recursive":
            splitter = RecursiveCharacterTextSplitter(
                chunk_size=chunk_size,
                chunk_overlap=chunk_overlap,
                separators=["\n\n", "\n", " ", ""]
            )
            chunks = splitter.split_documents(documents)
            
        elif strategy == 'markdown':
            header_splitter = MarkdownHeaderTextSplitter(
                headers_to_split_on=[
                    ("#", "Header 1"),
                    ("##", "Header 2"),
                    ("###", "Header 3"),
                ]
            )
            chunks = header_splitter.split_documents(documents)
        
        elif strategy == 'semantic':
            embeddings = OpenAIEmbeddings()
            text_splitter = SemanticChunker(
                embeddings=embeddings,
                breakpoint_threshold_type="percentile"
            )
            chunks = text_splitter.split_documents(documents)
        
        elif strategy == 'context-aware':
            splitter = RecursiveCharacterTextSplitter(
                chunk_size=chunk_size, 
                chunk_overlap=chunk_overlap, 
                separators=["\n\n", "\n", " ", ""]
            )
            chunks = splitter.split_documents(documents)
            
            # Add context description using LLM
            prompt_template = PromptTemplate.from_template(
"""You are an expert at summarizing information for retrieval.
                
Analyze the following text chunk and provide a brief, high-level context description.
Focus on the main topic, entities, or event described.

Text Chunk:
"{chunk_text}"

Context Description:"""
            )
            
            llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=1.0)
            chain = prompt_template | llm | StrOutputParser()
            
            for chunk in tqdm(chunks):
                context = chain.invoke({'chunk_text': chunk.page_content})
                chunk.metadata['context_description'] = context.strip()
        
        else:
            raise ValueError(f"Unknown chunking strategy: {strategy}")
        
        return chunks

In [8]:
chunked_docs = chunk_documents(documents, strategy='recursive', chunk_size=500, chunk_overlap=50)

In [9]:
print(f"Number of chunks created: {len(chunked_docs)}")

Number of chunks created: 66


In [10]:
for i, doc in enumerate(chunked_docs[:3]):
    print(f"Chunk {i}: {doc.page_content}")
    print("-" * 80)

Chunk 0: Chính sách Quyền riêng tư của TechNova Solutions



Cập nhật lần cuối: Ngày 20 tháng 12 năm 2025



Chào mừng bạn đến với TechNova Solutions ("Công ty," "chúng tôi," hoặc "của chúng tôi"). Chúng tôi cam kết bảo vệ thông tin cá nhân và quyền riêng tư của bạn. Nếu bạn có bất kỳ câu hỏi hoặc lo ngại nào về chính sách của chúng tôi, hoặc về các thực tiễn của chúng tôi liên quan đến thông tin cá nhân của bạn, vui lòng liên hệ với chúng tôi tại privacy@technovasolutions.io.
--------------------------------------------------------------------------------
Chunk 1: Khi bạn truy cập trang web của chúng tôi https://www.technovasolutions.io, sử dụng ứng dụng di động của chúng tôi, hoặc sử dụng bất kỳ dịch vụ nào của chúng tôi (gọi chung là "Dịch vụ," bao gồm Trang web và Ứng dụng), chúng tôi đánh giá cao việc bạn đã tin tưởng giao phó thông tin cá nhân của mình cho chúng tôi. Chúng tôi rất coi trọng quyền riêng tư của bạn. Trong thông báo quyền riêng tư này, chúng tôi cố gắng giải thích c

# 4. Store to Vector DB
    - Connect to DB
    - Get/Create collection (vectorizer, reranker, properties(for metadata filtering))
    - Load doc to collection

In [11]:
# Connect to Weaviate instance
client = weaviate.connect_to_local(
    port=8080, 
    grpc_port=50051
)

e:\Individual\Projects\project_meetly_flask\.venv\Lib\site-packages\weaviate\warnings.py:133: DeprecationWarning: Dep005: You are using weaviate-client version 4.10.2. The latest version is 4.19.0.
            Consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


In [12]:
collection_name = "ProjectDocuments"

# Check if collection exists
if client.collections.exists(collection_name):
    client.collections.delete(collection_name)
else:
    # Create new collection
    client.collections.create(
        name=collection_name,
        vectorizer_config=Configure.Vectorizer.text2vec_transformers(),
        reranker_config=Configure.Reranker.transformers(),
        properties=[
            Property(name='content', data_type=DataType.TEXT, vectorize_property=True),
            Property(name='source', data_type=DataType.TEXT, vectorize_property=False),
        ]
    )

In [13]:
collection = client.collections.get(collection_name)
# Inser documents into the collection
with collection.batch.dynamic() as batch:
    for doc in tqdm(chunked_docs):
        properties = {
            'content': doc.page_content,
            'source': doc.metadata.get('source', ''),
        }
        
        batch.add_object(
            properties=properties,
            uuid=generate_uuid5(doc.page_content)
        )

100%|██████████| 66/66 [00:00<00:00, 15708.11it/s]

In [14]:
# Check for errors
if len(collection.batch.failed_objects) > 0:
    print(f"Errors occurred: {len(collection.batch.failed_objects)} objects failed")
else:
    print("All documents stored successfully!")

All documents stored successfully!


In [ ]:
results = collection.query.fetch_objects(limit=3, include_vector=True)
for obj in results.objects:
    print("Properties:", obj.properties)
    print("Vectors:", obj.vector)

In [19]:
result = collection.aggregate.over_all(total_count=True)
print(f"Total objects in collection: {result.total_count}")

Total objects in collection: 57


In [15]:
client.close()